## Coding Exercise #0901

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Install tensorflow_datasets.
!pip install tensorflow_datasets

In [ ]:
# Load MNIST datasets.
import tensorflow_datasets as tfds

In [ ]:
# Set hyperparameters: BUFFER_SIZE to shuffle data, BATCH_SIZE that is mini-batch size, and the number of training epochs.
import numpy as np
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20

In [ ]:
# Define the number of times to repeat per epoch.
steps_per_epoch = np.ceil(60000 / BATCH_SIZE)

In [ ]:
# Define a function that preprocesses the input image and label.
def preprocess(item):
    image = item['image']
    label = item['label']
    image = tf.image.convert_image_dtype(image, tf.float32) # Convert to a tf.float32 type as it was originally a 'unit8' type with a range of [0, 255] 
    return {'image-pixels' : image}, label[... , tf.newaxis]

In [ ]:
# Define two input functions for training and evaluation.
def train_input_fn():
    datasets = tfds.load(name='mnist', )
    mnist_train = datasets['train']

    dataset = mnist_train.map(preprocess)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset.repeat()

In [ ]:
def eval_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_test = datasets['test']
    dataset = mnist_test.map(preprocess).batch(BATCH_SIZE)
    return dataset

In [ ]:
# Define a feature column.
image_feature_column = tf.feature_column.numeric_column(key='image-pixels', shape=(28*28)) # Make it as a feature column of 784

In [ ]:
# After defining an estimator object
dnn_classifier = tf.estimator.DNNClassifier(
    feature_columns=[image_feature_column],
    hidden_units=[32, 16],
    n_classes=10)

In [ ]:
# Training, evaluation, prediction
dnn_classifier.train(
    input_fn=train_input_fn,
    steps=NUM_EPOCHS * steps_per_epoch)

eval_result = dnn_classifier.evaluate(
    input_fn=eval_input_fn)

print(eval_result)